In [ ]:
BLACKHOLE = False


import os
import sys
from pathlib import Path


if BLACKHOLE:
    workspace_path = os.path.expandvars('$BLACKHOLE')
    sys.path.append(workspace_path+'/DeepLearning/02456_news_project/src')
    DATAPATH = Path(workspace_path+"/DeepLearning/ebnerd_data").expanduser()
else:
    DATAPATH = Path("~/ebnerd_data").expanduser()

DATASET = "ebnerd_demo"
# DATASET = "ebnerd_small"

In [2]:
import torch

print("torch version:", torch.__version__)

# Check gpu availability


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.zeros(1).cuda())

torch version: 2.5.1+cu124
cuda
tensor([0.], device='cuda:0')


In [4]:
# Check if the model can be loaded
from nrms import NRMSModel
from hyperparameters import HyperParameters

# hparams contains the hyperparameters of the model:
# They are:
# vocab_size: int
# word_embedding_dim: int
# dropout: float


hparams = HyperParameters(
    vocab_size=100, 
    word_embedding_dim=300, 
    dropout=0.1, 
    head_num=8,
    head_dim=64,
    attention_hidden_dim=512,
    title_size=20
)

word2vec_embedding = torch.randn(hparams.vocab_size, hparams.word_embedding_dim)

model = NRMSModel(hparams=hparams, word2vec_embedding=word2vec_embedding)

print(model)




NRMSModel(
  (word2vec_embedding): Embedding(100, 300)
  (dropout): Dropout(p=0.1, inplace=False)
  (news_encoder): Sequential(
    (0): Embedding(100, 300)
    (1): Dropout(p=0.1, inplace=False)
    (2): SelfAttention(
      (query_layer): Linear(in_features=64, out_features=512, bias=True)
      (key_layer): Linear(in_features=64, out_features=512, bias=True)
      (value_layer): Linear(in_features=64, out_features=512, bias=True)
    )
    (3): Dropout(p=0.1, inplace=False)
    (4): AttentionLayer2(
      (attention_weight): Linear(in_features=512, out_features=1, bias=True)
    )
  )
  (user_encoder): Sequential(
    (0): Linear(in_features=20, out_features=512, bias=True)
    (1): SelfAttention(
      (query_layer): Linear(in_features=64, out_features=512, bias=True)
      (key_layer): Linear(in_features=64, out_features=512, bias=True)
      (value_layer): Linear(in_features=64, out_features=512, bias=True)
    )
    (2): AttentionLayer2(
      (attention_weight): Linear(in_featu

TODO HERFRA OG NED

In [ ]:

from data_handler import NewsDataset

# test
dataset = NewsDataset(dataset_path=DATAPATH.joinpath(DATASET))

print(dataset.df_history_train.head())

print(dataset.df_behaviors_train.head())

shape: (5, 5)
┌─────────┬──────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┐
│ user_id ┆ impression_time_fixe ┆ scroll_percentage_f ┆ article_id_fixed    ┆ read_time_fixed     │
│ ---     ┆ d                    ┆ ixed                ┆ ---                 ┆ ---                 │
│ u32     ┆ ---                  ┆ ---                 ┆ list[i32]           ┆ list[f32]           │
│         ┆ list[datetime[μs]]   ┆ list[f32]           ┆                     ┆                     │
╞═════════╪══════════════════════╪═════════════════════╪═════════════════════╪═════════════════════╡
│ 13538   ┆ [2023-04-27          ┆ [100.0, 35.0, …     ┆ [9738663, 9738569,  ┆ [17.0, 12.0, …      │
│         ┆ 10:17:43, 2023-04-…  ┆ 100.0]              ┆ … 9769366]          ┆ 16.0]               │
│ 58608   ┆ [2023-04-27          ┆ [37.0, 61.0, …      ┆ [9739362, 9739179,  ┆ [2.0, 24.0, … 0.0]  │
│         ┆ 18:48:09, 2023-04-…  ┆ null]               ┆ … 9770333]          

In [11]:
# Use the dataloader to separate the data into batches / validation / test sets
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

for batch in dataloader:
    print(batch)
    break


TypeError: object of type 'NewsDataset' has no len()

In [ ]:
# Train the model

from train import train

train(
    model=model,
    dataloader=dataloader,
    hparams=hparams,
    device=device
)

# Save the model
torch.save(model.state_dict(), "model.pth")